# Configurações

![alt text](https://github.com/elthonf/plataformas-cognitivas-docker/blob/master/images/capa.png?raw=1)

In [11]:
#@title Parâmetros da solução
#@markdown Informe a quantidade de clientes que devem ser analisados.
qtde_clientes = 10  #@param {type: "slider", min: 1, max: 10}

#@markdown Informe o IP do servidor de predição da sua solução:
my_server_ip = "13.64.51.123" #@param {type:"string"}

#@markdown Informe a porta do servidor de predição da sua solução, se nada foi alterado, deve ser 443:
my_server_port = "443" #@param {type:"string"}

#@markdown Informe o seu nome:
students_name = "Elthon Manhas de Freitas" #@param {type:"string"}

#@markdown Ah, e não se esqueça de executar esta célula!


In [12]:
import requests
import pandas as pd
import json

# Obtenção dos empréstimos pendentes de análise

Vamos agora obter os clientes que estã precisand de empréstimo e exibí-los na tela

In [13]:
url = "https://us-central1-emf-teacher.cloudfunctions.net/function-aulas-getclient?qtde={0}".format(qtde_clientes)
headers = {'Content-Type': 'application/json'}
response = requests.request("GET", url, headers=headers)
response_content = response.text.encode('utf8').decode()
clientes = pd.read_json(response_content)

<ipython-input-13-c92bf7d9a343>:5: FutureWarning: Passing literal json to 'read_json' is deprecated and will be removed in a future version. To read from a literal string, wrap it in a 'StringIO' object.
  clientes = pd.read_json(response_content)


In [14]:
clientes

,nome,renda,idade,etnia,sexo,casapropria,outrasrendas,estadocivil,escolaridade
3300,"Nicole, Russo",1995.421750,18.086341,0,1,0,0,1,2
9651,"Jaron, Williams",3346.065855,41.151762,1,0,1,1,1,2
10994,"Jared, Le",4638.895226,58.086037,0,0,1,1,0,1
26082,"Corey, Martzloff",4001.890894,25.776492,1,0,1,0,1,2
35437,"Reginald, Lipscomb",4857.156856,47.581025,1,0,0,0,1,3
7412,"Abdullah, el-Jamil",4351.564010,44.204447,0,0,1,0,1,3
36456,"Shelbee, Stevenson",3169.630893,45.491622,0,1,0,0,0,1
29646,"Anthony, Hanson",3706.536663,52.101908,0,0,1,1,0,2
48001,"Lindsay, Wahl",2897.291111,21.122773,1,1,0,0,1,0
7653,"Geovanni, Lopez",3944.752064,54.201881,1,0,0,0,1,3


# Predição

O cliente agora será submetido a dois modelos de Machine Learning preparados para a análise.

In [15]:
url = "http://{}:{}/predict".format(my_server_ip, my_server_port)
headers = {'Content-Type': 'application/json'}
conteudo = clientes.to_json()

In [16]:
response01 = requests.request("POST", url + f"?model=modelo01&aluno=a", headers=headers, data=conteudo)
respostas01 = json.loads(response01.text.encode('utf8').decode())
respostas01

{'prediction': [1, 0, 0, 0, 0, 0, 1, 0, 1, 0],
 'proba': [[0.00013698872951228847, 0.9998630112704877],
  [0.9405399427875103, 0.05946005721248969],
  [0.9999685485098961, 3.145149010391654e-05],
  [0.8730746261289017, 0.1269253738710982],
  [0.9988039866173307, 0.0011960133826693206],
  [0.9917593122633227, 0.00824068773667735],
  [0.05202654389728534, 0.9479734561027147],
  [0.9844670793907774, 0.01553292060922262],
  [0.021383447018678003, 0.978616552981322],
  [0.6690505015622995, 0.3309494984377005]]}

In [17]:
df_01 = pd.Series(respostas01['prediction'], index = clientes.index, name="PredicaoML01")
df_01B = pd.Series(respostas01['proba'], index = clientes.index, name="PredicaoML01B")

In [18]:
response02 = requests.request("POST", url + "?model=modelo02", headers=headers, data=conteudo)
respostas02 = json.loads(response02.text.encode('utf8').decode())
respostas02

{'prediction': [0.9968073961632616,
  0.03695219263821489,
  -0.04781701770791742,
  0.050057400418944534,
  -0.009887110344211959,
  0.0367415766740399,
  0.9832239191772562,
  0.0005934691844828709,
  1.0317622849396224,
  0.1851815900153027]}

In [19]:
df_02 = pd.Series(respostas02['prediction'], index = clientes.index, name="PredicaoML02")

In [20]:
pd.DataFrame(clientes.loc[:,"nome"])\
    .merge(right= df_01, left_index=True, right_index=True)\
    .merge(df_01B, left_index=True, right_index=True)\
    .merge(df_02, left_index=True, right_index=True)

,nome,PredicaoML01,PredicaoML01B,PredicaoML02
3300,"Nicole, Russo",1,"[0.00013698872951228847, 0.9998630112704877]",0.996807
9651,"Jaron, Williams",0,"[0.9405399427875103, 0.05946005721248969]",0.036952
10994,"Jared, Le",0,"[0.9999685485098961, 3.145149010391654e-05]",-0.047817
26082,"Corey, Martzloff",0,"[0.8730746261289017, 0.1269253738710982]",0.050057
35437,"Reginald, Lipscomb",0,"[0.9988039866173307, 0.0011960133826693206]",-0.009887
7412,"Abdullah, el-Jamil",0,"[0.9917593122633227, 0.00824068773667735]",0.036742
36456,"Shelbee, Stevenson",1,"[0.05202654389728534, 0.9479734561027147]",0.983224
29646,"Anthony, Hanson",0,"[0.9844670793907774, 0.01553292060922262]",0.000593
48001,"Lindsay, Wahl",1,"[0.021383447018678003, 0.978616552981322]",1.031762
7653,"Geovanni, Lopez",0,"[0.6690505015622995, 0.3309494984377005]",0.185182
